In [59]:
import csv
import fileinput
import csv
import glob
import os
import pandas as pd
directory = os.getcwd()

#outputs to a folder
output = input("OUTPUT Folder: ")
output = os.path.join(os.getcwd(),output)

#create the folder if none exist
if not os.path.exists(output):
    os.makedirs(output)

#finds all text files to convert
txt_files = os.path.join(directory, '*.txt')

for txt_file in glob.glob(txt_files):
    with open(txt_file, "r") as input_file:
        in_txt = csv.reader(input_file, delimiter = '\t')
        filename = os.path.splitext(os.path.basename(txt_file))[0] + '.csv'
        
        #strips whitespace
        aList=[]
        for row in in_txt:
            aList.append([cell.rstrip() for cell in row])
        
        with open(os.path.join(output, filename), 'w') as output_file:
            out_csv = csv.writer(output_file)
            out_csv.writerows(aList)

df = pd.read_csv('csv/NBA Hackathon - Event Codes.csv')
df2 = pd.read_csv('csv/NBA Hackathon - Play by Play Data Sample (50 Games).csv')

output = pd.merge(df,df2,on=['Event_Msg_Type','Action_Type'])

removeFromList = ['Jump Ball', 'Missed Shot', 'Rebound', 'Timeout','Turnover','Violation']
for event in removeFromList:
    output=output[output['Event_Msg_Type_Description']!= event]
output.reset_index(drop=True, inplace = True)


df = pd.read_csv('csv/NBA Hackathon - Game Lineup Data Sample (50 Games).csv')
df = df[['Game_id','Person_id','Team_id']]
df2 = output
# df2.drop('Unnamed: 0', axis = 1, inplace=True)
df_p1 = df.drop_duplicates()
df_p2 = df_p1.copy()
df_p1.columns = ['Game_id','Person1','Team_id_check']
df_p2.columns = ['Game_id','Person2','Team_id_check2']
output = pd.merge(df_p1,df2,on=['Game_id','Person1'],how = 'outer')
output = pd.merge(df_p2,output,on=['Game_id','Person2'],how = 'outer')
output = output.sort_values(by=['Game_id','Event_Num'])
pd.options.display.float_format = '{:,.0f}'.format


for index,series in output.iterrows():
        if pd.isnull(series['Team_id_check']) and series['Event_Msg_Type'] == 8:
            output.loc[index, 'Team_id_check'] = series['Team_id_check2']
        if pd.isnull(series['Team_id_check']) and series['Event_Msg_Type'] == 1:
            output.loc[index, 'Team_id_check'] = series['Team_id']
        if pd.isnull(series['Team_id_check']) and series['Event_Msg_Type'] == 6:
            output.loc[index, 'Team_id_check'] = series['Team_id']
        if pd.isnull(series['Team_id_check']) and series['Event_Msg_Type'] == 3:
            output.loc[index, 'Team_id_check'] = series['Team_id']
        if pd.isnull(series['Team_id_check']) and series['Event_Msg_Type'] == 8:
            output.loc[index, 'Team_id_check'] = series['Team_id']
colsOrder = ['Game_id','Event_Num','Event_Msg_Type','Period','WC_Time','PC_Time','Action_Type','Option1','Option2','Option3','Team_id','Person1',\
             'Person2','Team_id_type', 'Event_Msg_Type_Description', 'Action_Type_Description','Team_id_check','Team_id_check2']
floats = ['Event_Num','Event_Msg_Type','Period','WC_Time','PC_Time','Action_Type','Option1','Option2','Option3','Team_id_type']
output=output[colsOrder]
output.dropna(subset=['Event_Num'],inplace=True)
output[floats] = output[floats].astype(int)
output.reset_index(drop=True).to_csv('data.csv')


OUTPUT Folder: CSV


In [ ]:
'Event_Num','Event_Msg_Type','Period','WC_Time','PC_Time','Action_Type','Option1','Option2','Option3','Team_id_type'